# Training a text classifier model with fastai
- this notebook assumes you have already run text_model_training.ipynb notebook
- In this notebook, the IMDB dataset is ingested
- the first section

In [1]:
#hide
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

In [2]:
#hide
from fastbook import *
from fastai.text.all import *

In [3]:
# switch to control whether direct TDL or DataBlocks definition used 
tdl = True

In [4]:
modifier = 'mar3'

# Ingest the dataset
- define the path for the dataset
- create a TextDataLoaders object

In [5]:
%%time
# create dataloaders object
path = untar_data(URLs.IMDB)
path.ls()

CPU times: user 4.86 ms, sys: 139 µs, total: 5 ms
Wall time: 93.4 ms


(#7) [Path('/storage/data/imdb/README'),Path('/storage/data/imdb/tmp_lm'),Path('/storage/data/imdb/imdb.vocab'),Path('/storage/data/imdb/tmp_clas'),Path('/storage/data/imdb/test'),Path('/storage/data/imdb/train'),Path('/storage/data/imdb/unsup')]

# Define the text classifier

In [6]:
%%time
# define TextDataLoaders object
dls_clas = TextDataLoaders.from_folder(path, valid='test')


CPU times: user 8.09 s, sys: 2.57 s, total: 10.7 s
Wall time: 10.8 s


In [7]:
# directory structure of the IMDB curated dataset
'''
├── test
│   ├── neg
│   └── pos
├── tmp_clas
├── tmp_lm
├── train
│   ├── neg
│   └── pos
└── unsup
'''

'\n├── test\n│   ├── neg\n│   └── pos\n├── tmp_clas\n├── tmp_lm\n├── train\n│   ├── neg\n│   └── pos\n└── unsup\n'

In [8]:
dls_clas.path

Path('/storage/data/imdb')

In [ ]:
dls_clas.show_batch(max_n=3)

In [9]:
# save the current path
keep_path = path
print("keep_path is: ",str(keep_path))

keep_path is:  /storage/data/imdb


In [10]:
%%time
# define a text_classifier_learner object
learn_clas = text_classifier_learner(dls_clas, AWD_LSTM, 
                                metrics=accuracy).to_fp16()

CPU times: user 8.08 s, sys: 1.53 s, total: 9.6 s
Wall time: 12.9 s


# Fine-tune the text classifier
Use the encoder created as part of training the language model to fine tune the text classifier

In [11]:
# Path('/storage/data/imdb')
learn_clas.path

Path('/storage/data/imdb')

In [12]:
%%time
# set the path to the location of the encoder
learn_clas.path = Path('/notebooks/temp')

CPU times: user 339 µs, sys: 80 µs, total: 419 µs
Wall time: 54.4 µs


In [13]:
# load the encoder that was saved when the language model was trained
learn_clas = learn_clas.load_encoder('ft_'+modifier)

In [14]:
learn_clas.path

Path('/notebooks/temp')

In [15]:
# set the path back to the original path
learn_clas.path = keep_path

In [16]:
# ch 10 style Path('/storage/data/imdb')
learn_clas.path

Path('/storage/data/imdb')

In [ ]:
%%time
# fine tune the model
learn_clas.fit_one_cycle(5, 2e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.431991,0.294341,0.876600,08:29
1,0.409600,0.281576,0.883800,04:39
2,0.396112,0.270241,0.888880,04:10
3,0.411736,0.263737,0.891000,05:10


In [ ]:
x, y = first(dls_clas.train)
x.shape, y.shape, len(dls_clas.train)

In [ ]:
learn_clas.summary()

# Exercise the text classifier
Apply the fine-tuned text classifier on some text samples.

In [ ]:
preds = learn_clas.predict("this film shows incredibly bad writing and is a complete disaster")

In [ ]:
preds

In [ ]:
preds = learn_clas.predict("this film shows incredible talent and is a complete triumph")

In [ ]:
preds

In [ ]:
# save the classifier model
learn_clas.path = Path('/notebooks/temp')
learn_clas.save('classifier_single_epoch_'+modifier+'d')